In [ ]:
import numpy as np
import scipy.io as io
import pandas as pd
import sys

import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold, StratifiedKFold


In [ ]:
X_all = data['X_all'] #n_trtype, n_conds,n_subj, n_time, n_elec 
subjects = data['subjects']
n_trtype, n_conds,n_subj, n_time, n_elec = X_all.shape


In [ ]:
test_pca1 = np.array([32])#np.arange(5,38,5)#np.array([15,38])#
test_pca2 = np.arange(2,n_subj,3) 
test_pca3 = np.array([15,38])

test_C1 =np.logspace(-5, 0, 6)
test_C2=np.logspace(-3, 0, 21)  #-3, 0, 41) 


In [ ]:
pca = PCA()
test_pca=test_pca2
test_C=test_C2

clfs={}

clfs['LR-L2']= LogisticRegression(max_iter=5000, tol=1.0) #default=’l2’
clfs['LR-L1']= LogisticRegression(penalty='l1', solver='liblinear', max_iter=5000, tol=1.0)
clfs['LDA']=  LinearDiscriminantAnalysis(tol=1e-3)
clfs['QDA'] = QuadraticDiscriminantAnalysis(tol=1e-3)


In [ ]:
clf_key_8x4='LR-L2'
pipe_8x4 = Pipeline(steps=[('pca', pca), ('clf', clfs[clf_key_8x4])])

clf_key_8x1='LR-L2'
pipe_8x1 = Pipeline(steps=[('pca', pca), ('clf', clfs[clf_key_8x1])])

param_grid = {'pca__n_components': test_pca,'clf__C': test_C}

print('Testing '+ str(param_grid['pca__n_components'].shape[0]) +' different n_pca and '+str(param_grid['clf__C'].shape[0]) +' regularization parameters')


In [ ]:
y=subjects[...,0]

In [ ]:
search_8x4 = np.zeros((n_trtype,n_conds), dtype=object)
for i in range(n_trtype):
    for j in range(n_conds):
        X = X_all[i,j].reshape(n_subj,n_time*n_elec)**2        
        #X -= X.mean(0, keepdims=True)
        # /= X.std(0, keepdims=True)
        search_8x4[i,j] = GridSearchCV(pipe_8x4, param_grid, cv=5, n_jobs=15, verbose=0)
        search_8x4[i,j].fit(X, y);
        verbose('%i,%i'%(i,j))

run_8x4_identifier=clf_key_8x4+'-pca'+str(param_grid['pca__n_components'].shape[0])+'-C'+str(param_grid['clf__C'].shape[0])+'-'+str(X.shape[1])+'features'
save_name='results-8x4-'+baseMethod+'-'+run_8x4_identifier+'.npz'  #date.today().strftime('--%d-%m-%Y')
np.savez(save_dir+save_name,search_8x4=search_8x4)


In [ ]:
search_8x1 = np.zeros(n_trtype, dtype=object)
for i in range(n_trtype):
    X = X_all[i].reshape(n_subj,n_time*n_elec*n_conds)
    search_8x1[i] = GridSearchCV(pipe_8x1, param_grid, cv=5, n_jobs=15, verbose=0)
    search_8x1[i].fit(X, y);
    verbose('%i'%(i))

run_8x1_identifier=clf_key_8x1+'-pca'+str(param_grid['pca__n_components'].shape[0])+'-C'+str(param_grid['clf__C'].shape[0])+'-'+str(X.shape[1])+'features'
save_name='results-8x1-'+baseMethod+'-'+run_8x1_identifier+'.npz'  #date.today().strftime('--%d-%m-%Y')
np.savez(save_dir+save_name,search_8x1=search_8x1)
    